In [1]:
import regex as re
import pandas as pd
import pdfplumber

In [2]:
# pdf = pdfplumber.open('DataUUHabibie/uu8-1999.pdf')
# page = pdf.pages[:]
# text = ''

# text += page[0].extract_text()

# for i in page[1:]:
#     text += "\n" + i.extract_text()

# file = text

In [3]:
# file_path = "UU_9_2013 (1).txt"

# try:
#     with open(file_path, encoding="utf8") as file_uu:
#         file = file_uu.read()
#         file_uu.close()
# except:
#     with open(file_path) as file:
#         file = file_uu.read()
#         file_uu.close()

In [4]:
no_uu  = 17

pdf_badan = pdfplumber.open(f'DataUUSBY/nomor_{no_uu}/uu{no_uu}-2013bt.pdf')
page = pdf_badan.pages[:]
text = ''

text += page[0].extract_text()

for i in page[1:]:
    text += "\n" + i.extract_text()

file_badan = text


pdf_penjelasan = pdfplumber.open(f'DataUUSBY/nomor_{no_uu}/uu{no_uu}-2013pjl.pdf')
page = pdf_penjelasan.pages[:]
text = ''

text += page[0].extract_text()

for i in page[1:]:
    text += "\n" + i.extract_text()

file_penjelasan = text

In [5]:
pdf_badan.close()
pdf_penjelasan.close()

## Remove Header

In [6]:
# file_no_header = re.sub("\\nPRESIDEN\\nREPUBLIK INDONESIA\\n- \p{N}+ -", " ", file, flags=re.DOTALL)
# file_no_header = re.sub("(\\n[^\\n]*? \…[^\\n]*)(?=\\n)", " ", file_no_header, flags=re.DOTALL)

In [7]:
badan = re.sub("\\nPRESIDEN\\nREPUBLIK INDONESIA\\n- \p{N}+ -", " ", file_badan, flags=re.DOTALL)
badan = re.sub("(\\n[^\\n]*? \…[^\\n]*)(?=\\n)", " ", badan, flags=re.DOTALL)
badan = re.sub("(www.*?id\\n.*?\\n|www.*?id)", " ", badan, flags=re.DOTALL)
badan = re.sub("(\\n\s+|\s+\\n)", "\\n", badan, flags=re.DOTALL)
penjelasan = re.sub("\\nPRESIDEN\\nREPUBLIK INDONESIA\\n- \p{N}+ -", " ", file_penjelasan, flags=re.DOTALL)
penjelasan = re.sub("(\\n[^\\n]*? \…[^\\n]*)(?=\\n)", " ", penjelasan, flags=re.DOTALL)
penjelasan = re.sub("(www.*?id\\n.*?\\n|www.*?id)", " ", penjelasan, flags=re.DOTALL)
penjelasan = re.sub("(\\n\s+|\s+\\n)", "\\n", penjelasan, flags=re.DOTALL)

In [8]:
# file_no_header

## Pisah Penjelasan

In [9]:
# penjelasan = re.search("(Agar setiap orang mengetahuinya.*|Agar setiap orang dapat mengetahuinya.*)", file_no_header, re.DOTALL).group()
# badan = file_no_header.replace(penjelasan, "")

In [10]:
# penjelasan = re.sub("TAMBAHAN LEMBARAN NEGARA.*", "", penjelasan, flags=re.DOTALL)

# Badan

In [11]:
badan

'LEMBARAN NEGARA\nREPUBLIK INDONESIA\nNo.116, 2013 HAK ASASI MANUSIA. Organisasi\nKemasyarakatan. Pendirian-Pengawasan.\nPencabutan. (Penjelasan Dalam Tambahan\nLembaran Negara Republik Indonesia Nomor\n5430)\nUNDANG-UNDANG REPUBLIK INDONESIA\nNOMOR 17 TAHUN 2013\nTENTANG\nORGANISASI KEMASYARAKATAN\nDENGAN RAHMAT TUHAN YANG MAHA ESA\nPRESIDEN REPUBLIK INDONESIA,\nMenimbang : a. bahwa kebebasan berserikat, berkumpul, dan\nmengeluarkan pendapat merupakan bagian dari hak\nasasi manusia dalam kehidupan berbangsa dan\nbernegara dalam Negara Kesatuan Republik\nIndonesia yang dijamin oleh Undang-Undang Dasar\nNegara Republik Indonesia Tahun 1945;\nb. bahwa dalam menjalankan hak dan kebebasan\nberserikat, berkumpul, dan mengeluarkan pendapat,\nsetiap orang wajib menghormati hak asasi dan\nkebebasan orang lain dalam rangka tertib hukum\nserta menciptakan keadilan dalam kehidupan\nbermasyarakat, berbangsa, dan bernegara;\nc. bahwa sebagai wadah dalam menjalankan\nkebebasan berserikat, berkumpul,

## Ambil BAB

In [12]:
def ambil_bab(text):
    pattern = "(?<=\\n)(BAB\s*[IVXLCDM]+.*?)(?=\\n\s*BAB|$)"
    bab = re.findall(pattern, text, re.DOTALL)
    if bab:
        ## ambil nomor dan nama bab
        pattern_no = "(?<=BAB)(?:\s*)([IVXLCDM]+)"
        pattern_nama = "(?<=BAB)(?:\s*[IVXLCDM]+\s*\\n)(.*?)(?=\\n\p{Lu}\p{Ll})"
        pattern_isi = "(?<=BAB)(?:\s*[IVXLCDM]+\s*\\n.*?)(?:\\n)(\p{Lu}\p{Ll}.*)"
        no = [re.search(pattern_no, i, re.DOTALL).group(1) if re.search(pattern_no, i, re.DOTALL) else "ERROR" for i in bab]
        nama = [re.search(pattern_nama, i, re.DOTALL).group(1) if re.search(pattern_nama, i, re.DOTALL) else "ERROR" for i in bab]
        isi = [re.search(pattern_isi, i, re.DOTALL).group(1) if re.search(pattern_isi, i, re.DOTALL) else "ERROR" for i in bab]
        return no, nama, isi
    else:
        no = [""]
        nama = [""]
        isi = [text]
        return no, nama, isi

Penjelasan
1. (?<=\\n)(BAB [IVXLCDM]+.*?)(?=\\nBAB|$)<br>
Digunakan untuk mengambil semua bab<br>
Kondisi:<br>
    a. Text dimulai dengan \n tanpa mengambil \n<br>
    b. Setelah \n dimulai dengan BAB(spasi)[Bilangan Romawi 1 digit dapat diulang beberapa kali] contoh (BAB IIV, BAB VVIX) akan diambil<br>
       BILANGAN ROMAWI YANG TIDAK VALID DAPAT DIAMBIL<br>
    c. Setelah kondisi diatas akan mengambil semua karakter<br>
    d. pengambilan karakter akan berhenti ketika bertemu \nBAB<br>

2. (?<=BAB)(?:\s*)([IVXLCDM]+)<br>
Digunakan untuk mengambil no bab<br>
Kondisi:<br>
    a. Text dimulai dengan BAB(spasi dapat diulang beberapa kali) tanpa diambil<br>
    b. Setelah kondisi diatas [Bilangan romawi 1 digit dapat diulang beberapa kali] akan diambil<br>
       BILANGAN ROMAWI YANG TIDAK VALID DAPAT DIAMBIL<br>

3. (?<=BAB)(?:\s*[IVXLCDM]+\s*\\n)([\p{Lu}\s\\n]*)(?=\\n)<br>
Digunakan untuk mengambil nama bab<br>
Kondisi:<br>
    a. Text dimulai dengan BAB(spasi dapat diulang beberapa kali)[Bilangan Romawi 1 digit dapat diulang beberapa kali](spasi dapat diulang beberapa kali)\n<br>
    b. Mengambil semua karakter berupa huruf kapital atau spasi atau \n<br>
    c. Pengambilan karakter akan berhenti ketika bertemu \n<br>

4. (?<=BAB)(?:\s*[IVXLCDM]+\s*\\n[\p{Lu}\s\\n]*)(?:\\n)(.*)<br>
Digunakan untuk mengambil isi bab<br>
Kondisi:<br>
    a. Text dimulai dengan BAB(spasi dapat diulang beberapa kali)[Bilangan Romawi 1 digit dapat diulang beberapa kali](spasi dapat diulang beberapa kali)\n<br>
    b. Setelah kondisi di atas teks dimulai dengan huruf kapital atau spasi atau \n hingga berhenti pada \n tanpa diambil (nama bab)<br>
    c. Semua karakter sesudahnya diambil<br>

## Ambil Bagian

In [13]:
def ambil_bagian(text):
    pattern = "(?<=\\n|^)(Bagian\s*\p{Lu}.*?)(?=\\nBagian\s*\p{Lu}|$)"
    bagian = re.findall(pattern, text, re.DOTALL)
    if bagian:
        pattern_no = "(?<=Bagian)(?:\s*)(\p{Lu}\p{L}*\s*)(?=\\n)"
        pattern_nama = "(?<=Bagian)(?:\s*\p{Lu}\p{L}*\s*\\n)(.*?)(?=\\n(Pasal|Paragraf))"
        pattern_isi = "(Pasal|Paragraf).*"
        no = [re.search(pattern_no, i, re.DOTALL).group(1) if re.search(pattern_no, i, re.DOTALL) else "ERROR" for i in bagian]
        nama = [re.search(pattern_nama, i, re.DOTALL).group(1) if re.search(pattern_nama, i, re.DOTALL) else "ERROR" for i in bagian]
        isi = [re.search(pattern_isi, i, re.DOTALL).group(0) if re.search(pattern_isi, i, re.DOTALL) else "ERROR" for i in bagian]
        return no, nama, isi
    else:
        no = [""]
        nama = [""]
        isi = [text]
        return no, nama, isi

Penjelasan
1. (?<=\\n|^)(Bagian \p{Lu}.*?)(?=\\nBagian \p{Lu}|$)<br>
Digunakan untuk mengambil semua bagian<br>
Kondisi:<br>
    a. Text dimulai dengan \n atau (start of string) tanpa diambil<br>
    b. Setelah kondisi diatas dimulai dengan Bagian(spasi)[Huruf kapital] contoh (Bagian H, Bagian C, Bagian D) akan diambil<br>
    c. Setelah kondisi diatas akan mengambil semua karakter<br>
    d. pengambilan karakter akan berhenti ketika bertemu \nBagian(spasi)[Huruf kapital]<br>

2. (?<=Bagian \p{Lu})(?:\p{L}*\s*\\n)(.*?)(?=\\nPasal)<br>
Digunakan untuk mengambil nama bagian<br>
Kondisi:<br>
    a. Text dimulai dengan Bagian(spasi)[Huruf kapital][Huruf atau spasi dapat diulang beberapa kali]\n tanpa diambil<br>
    b. Semua karakter selanjutnya akan diambil<br>
    c. Pengambilan karakter akan berhenti ketika bertemu \nPasal<br>

3. Pasal.*<br>
Digunakan untuk mengambil isi bagian<br>
Kondisi:<br>
    a. Text dimulai dengan Pasal dengan diambil<br>
    b. Semua karakter selanjutnya akan diambil<br>

## Ambil Paragraf

In [14]:
def ambil_paragraf(text):
    pattern = "(?<=\\n|^)(Paragraf\s*\p{N}*.*?)(?=\\Paragraf\s*\p{N}|$)"
    paragraf = re.findall(pattern, text, re.DOTALL)
    if paragraf:
        pattern_no = "(?<=Paragraf)(?:\s*)(\p{N}*\s*)(?=\\n)"
        pattern_nama = "(?<=Paragraf)(?:\s*\p{N}*\s*\\n)(.*?)(?=\\nPasal)"
        pattern_isi = "Pasal.*"
        no = [re.search(pattern_no, i, re.DOTALL).group(1) if re.search(pattern_no, i, re.DOTALL) else "ERROR" for i in paragraf]
        nama = [re.search(pattern_nama, i, re.DOTALL).group(1) if re.search(pattern_nama, i, re.DOTALL) else "ERROR" for i in paragraf]
        isi = [re.search(pattern_isi, i, re.DOTALL).group(0) if re.search(pattern_isi, i, re.DOTALL) else "ERROR" for i in paragraf]
        return no, nama, isi
    else:
        no = [""]
        nama = [""]
        isi = [text]
        return no, nama, isi

## Ambil Pasal

In [15]:
def ambil_pasal(text):
    pattern = "(?<=\\n|^)(Pasal\s*[\p{N}IVXLCDM]+\s*\\n.*?)(?=\\nPasal\s*[\p{N}IVXLCDM]+\s*\\n|$)"
    pasal = re.findall(pattern, text, re.DOTALL)
    if pasal:
        pattern_no = "(?<=Pasal)(?:\s*)([\p{N}IVXLCDM]+)"
        pattern_isi = "(?<=Pasal)(?:\s*[\p{N}IVXLCDM]+\s*\\n)(.*)"
        no = [re.search(pattern_no, i, re.DOTALL).group(1) if re.search(pattern_no, i, re.DOTALL) else "ERROR" for i in pasal]
        isi = [re.search(pattern_isi, i, re.DOTALL).group(1) if re.search(pattern_isi, i, re.DOTALL) else "ERROR" for i in pasal]
        return no, isi
    else:
        no = [""]
        isi = [text]
        return no, isi

Penjelasan
1. (?<=\\n|^)(Pasal \p{N}+\s*\\n.*?)(?=\\nPasal|$)<br>
Digunakan untuk mengambil semua pasal<br>
Kondisi:<br>
    a. Text dimulai dengan \n atau (start of string) tanpa diambil<br>
    b. Setelah kondisi diatas dimulai dengan Pasal(spasi)[Nomor satu digit dapat diulang beberapa kali](spasi dapat diulang beberapa kali)\n contoh (Pasal 1\n, Pasal 23     \n) akan diambil<br>
    c. Setelah kondisi diatas akan mengambil semua karakter<br>
    d. Pengambilan karakter akan berhenti ketika bertemu \nPasal atau (end of string)<br>

2. (?<=Pasal)(?:\s*)(\p{N}+)<br>
Digunakan untuk mengambil no pasal<br>
Kondisi:<br>
    a. Text dimulai dengan Pasal(spasi dapat diulang beberapa kali) tanpa diambil<br>
    b. Setelah kondisi diatas dimulai dengan [Nomor satu digit dapat diulang beberapa kali] akan diambil<br>

3. (?<=Pasal)(?:\s*\p{N}+\s*\\n)(.*)<br>
Digunakan untuk mengambil isi pasal<br>
Kondisi:<br>
    a. Teks dimulai dengan Pasal(spasi dapat diulang beberapa kali) tanpa diambil<br>
    b. Setelah kondisi diatas dimulai dengan [Nomor satu digit dapat diulang beberapa kali](spasi dapat diulang beberapa kali)\n akan diambil (nomor pasal)<br>
    c. Setelah kondisi diatas akan mengambil semua karakter<br>

## Ambil Ayat

In [16]:
def ambil_ayat(text):
    clean_text = re.sub('(?<=ayat)(\\n)(?=\(\p{N}\))', "", text, flags=re.DOTALL)
    ayat = re.findall('(?<=\\n|^)(\(\p{N}+\).*?)(?=\\n\(\p{N}\)|$)', clean_text, flags=re.DOTALL)
    
    if ayat:
        pattern_no = "(?<=\\n|^)(?:\()(\p{N}+)(?=\))"
        pattern_isi = "(?<=\(\p{N}+\)\s*)(.*)"
        no = [re.search(pattern_no, i, re.DOTALL).group(1) if re.search(pattern_no, i, re.DOTALL) else "ERROR" for i in ayat]
        isi = [re.search(pattern_isi, i, re.DOTALL).group(0) if re.search(pattern_isi, i, re.DOTALL) else "ERROR" for i in ayat]
        
        return no, isi
    else:
        no = [""]
        isi = [text]
        return no, isi

Penjelasan
1. (?<=\\n|^)(\(\p{N}+\).*?)(?=\\n\(\p{N}\)|$)<br>
Digunakan untuk mengambil semua ayat<br>
Kondisi:<br>
    a. Text dimulai dengan \n atau (start of string) tanpa diambil<br>
    b. Setelah kondisi diatas dimulai oleh (buka kurung)[nomor dapat diulang beberapa kali](tutup kurung)<br>
    c. Setelah kondisi diatas semua karakter diambil<br>
    d. Pengambilan karakter diambil hingga bertemu \n(buka kurung)[nomor dapat diulang beberapa kali](tutup kurung) atau (end of string)<br>

2. (?<=\\n|^)(?:\()(\p{N}+)(?=\))<br>
Digunakan untuk mengambil no ayat<br>
Kondisi:<br>
    a. Text dimulai dengan \n atau (start of string) tanpa diambil<br>
    b. Setelah kondisi diatas dimulai oleh (buka kurung) tanpa diambil<br>
    c. Setelah kondisi diatas dimulai oleh [nomor dapat diulang beberapa kali] diambil<br>
    d. Pengambilan karakter diambil hingga bertemu (tutup kurung)<br>

3. (?<=\(\p{N}+\)\s*)(.*)<br>
Digunakan untuk mengambil isi ayat<br>
Kondisi:<br>
    a. Text dimulai dengan (buka kurung)[nomor dapat diulang beberapa kali](tutup kurung) tanpa diambil (no ayat)<br>
    b. Setelah kondisi diatas semua karakter diambil<br>

# Penjelasan

In [17]:
penjelasan

'TAMBAHAN\nLEMBARAN NEGARA RI\nNo. 5430 HAK ASASI MANUSIA. Organisasi\nKemasyarakatan. Pendirian-Pengawasan.\nPencabutan. (Penjelasan Atas Lembaran Negara\nRepublik Indonesia Tahun 2013 Nomor 116)\nPENJELASAN\nATAS\nUNDANG-UNDANG REPUBLIK INDONESIA\nNOMOR 17 TAHUN 2013\nTENTANG\nORGANISASI KEMASYARAKATAN\nI. UMUM\nUndang-Undang Dasar Negara Republik Indonesia Tahun 1945\nmenjamin kemerdekaan berserikat, berkumpul, dan mengeluarkan\npendapat serta memajukan dirinya dalam memperjuangkan haknya\nsecara individu ataupun kolektif untuk membangun masyarakat,\nbangsa, dan Negara Kesatuan Republik Indonesia sebagai perwujudan\nhak asasi manusia. Pasal 28J ayat (2) Undang-Undang Dasar Negara\nRepublik Indonesia Tahun 1945 menyebutkan bahwa dalam\nmenjalankan hak asasi dan kebebasannya secara individu maupun\nkolektif, setiap orang wajib menghormati hak asasi manusia lainnya\ndan wajib tunduk kepada pembatasan yang ditetapkan undang-\nundang dengan maksud semata-mata untuk menjamin pengakuan\nse

## Ambil Pasal

In [18]:
def ambil_penjelasan_pasal(text):
    # pattern = "(?<=\\n|^|\s)(Pasal\s*[\p{N}IVXLCDM]+\s*\\n.*?)(?=\\nPasal \p{N}+\s*\\n|\sPasal\s*[\p{N}IVXLCDM]+\s*\\n|$)"
    pattern = "(?<=\\n)(Pasal\s*[\p{N}IVXLCDM]+\s*\\n.*?)(?=\\nPasal\s*\p{N}+\s*\\n|$)"
    pasal = re.findall(pattern, text, re.DOTALL)
    if pasal:
        pattern_no = "(?<=Pasal)(?:\s*)([\p{N}IVXLCDM]+)"
        pattern_isi = "(?<=Pasal)(?:\s*[\p{N}IVXLCDM]+\s*\\n)(.*)"
        no = [re.search(pattern_no, i, re.DOTALL).group(1) if re.search(pattern_no, i, re.DOTALL) else "ERROR" for i in pasal]
        isi = [re.search(pattern_isi, i, re.DOTALL).group(1) if re.search(pattern_isi, i, re.DOTALL) else "ERROR" for i in pasal]
        return no, isi
    else:
        no = [""]
        isi = [text]
        return no, isi

## Ambil Ayat

In [19]:
def ambil_penjelasan_ayat(text):
    # pattern = "(?<=\\n|^|\s)(Ayat\s*\(\p{N}+\).*?)(?=\\nAyat\s*\(\p{N}+\)|\sAyat\s*\(\p{N}+\)|$)"
    pattern = "(?<=\\n|^)(\s*Ayat\s*\(\p{N}+\).*?)(?=\\nAyat\s*\(\p{N}+\)|$)"
    ayat = re.findall(pattern, text, re.DOTALL)
    if ayat:
        pattern_no = "(?<=Ayat)(?:\s*\()(\p{N}+)"
        pattern_isi = "(?<=Ayat)(?:\s*\(\p{N}+\)\s*\\n)(.*)"
        no = [re.search(pattern_no, i, re.DOTALL).group(1) if re.search(pattern_no, i, re.DOTALL) else "ERROR" for i in ayat]
        isi = [re.search(pattern_isi, i, re.DOTALL).group(1) if re.search(pattern_isi, i, re.DOTALL) else "ERROR" for i in ayat]
        return no, isi
    else:
        no = [""]
        isi = [text]
        return no, isi

In [20]:
df_badan_lst = []
lst_no_bab, lst_nama_bab, lst_isi_bab = ambil_bab(badan)

for i_bab, no_bab in enumerate(lst_no_bab):
    nama_bab = lst_nama_bab[i_bab]
    lst_no_bagian, lst_nama_bagian, lst_isi_bagian = ambil_bagian(lst_isi_bab[i_bab])

    for i_bagian, no_bagian in enumerate(lst_no_bagian):
        nama_bagian = lst_nama_bagian[i_bagian]
        lst_no_paragraf, lst_nama_paragraf, lst_isi_paragraf = ambil_paragraf(lst_isi_bagian[i_bagian])

        for i_paragraf, no_paragraf in enumerate(lst_no_paragraf):
            nama_paragraf = lst_nama_paragraf[i_paragraf]
            lst_no_pasal, lst_isi_pasal = ambil_pasal(lst_isi_paragraf[i_paragraf])

            for i_pasal, no_pasal in enumerate(lst_no_pasal):
                lst_no_ayat, lst_isi_ayat = ambil_ayat(lst_isi_pasal[i_pasal])
                df_ayat = pd.DataFrame()
                df_ayat["no_ayat"] = lst_no_ayat
                df_ayat["isi_ayat"] = lst_isi_ayat
                df_ayat["no_bab"] = no_bab
                df_ayat["nama_bab"] = nama_bab
                df_ayat["no_bagian"] = no_bagian
                df_ayat["nama_bagian"] = nama_bagian
                df_ayat["no_paragraf"] = no_paragraf
                df_ayat["nama_paragraf"] = nama_paragraf
                df_ayat["no_pasal"] = no_pasal
                df_badan_lst.append(df_ayat)

df_badan = pd.concat(df_badan_lst)
df_badan = df_badan[["no_bab", "nama_bab", "no_bagian", "nama_bagian", "no_paragraf", "nama_paragraf", "no_pasal", "no_ayat", "isi_ayat"]]

In [21]:
df_badan

,no_bab,nama_bab,no_bagian,nama_bagian,no_paragraf,nama_paragraf,no_pasal,no_ayat,isi_ayat
0,I,KETENTUAN UMUM,,,,,1,,Dalam Undang-Undang ini yang dimaksud dengan:\...
0,II,"ASAS, CIRI, DAN SIFAT",,,,,2,,Asas Ormas tidak bertentangan dengan Pancasila...
0,II,"ASAS, CIRI, DAN SIFAT",,,,,3,,Ormas dapat mencantumkan ciri tertentu yang me...
0,II,"ASAS, CIRI, DAN SIFAT",,,,,4,,"Ormas bersifat sukarela, sosial, mandiri, nirl..."
0,III,"TUJUAN, FUNGSI, DAN RUANG LINGKUP",,,,,5,,Ormas bertujuan untuk:\na. meningkatkan partis...
...,...,...,...,...,...,...,...,...,...
0,XVIII,KETENTUAN PERALIHAN,,,,,83,,Pada saat Undang Undang ini mulai berlaku:\na....
0,XIX,KETENTUAN PENUTUP,,,,,84,,"Pada saat Undang-Undang ini mulai berlaku, sem..."
0,XIX,KETENTUAN PENUTUP,,,,,85,,"Pada saat Undang-Undang ini mulai berlaku, Und..."
0,XIX,KETENTUAN PENUTUP,,,,,86,,Peraturan pelaksanaan dari Undang-Undang ini h...


In [22]:
df_penjelasan_lst = []

lst_no_penjelasan_pasal, lst_isi_penjelasan_pasal = ambil_penjelasan_pasal(penjelasan)

for i_penjelasan_pasal, no_penjelasan_pasal in enumerate(lst_no_penjelasan_pasal):
    lst_no_penjelasan_ayat, lst_isi_penjelasan_ayat = ambil_penjelasan_ayat(lst_isi_penjelasan_pasal[i_penjelasan_pasal])
    df_penjelasan_ayat = pd.DataFrame()
    df_penjelasan_ayat['penjelasan'] = lst_isi_penjelasan_ayat
    df_penjelasan_ayat['penjelasan_pasal'] = lst_isi_penjelasan_pasal[i_penjelasan_pasal]
    df_penjelasan_ayat['no_ayat'] = lst_no_penjelasan_ayat
    df_penjelasan_ayat['no_pasal'] = no_penjelasan_pasal
    df_penjelasan_lst.append(df_penjelasan_ayat)

df_penjelasan = pd.concat(df_penjelasan_lst)
df_penjelasan = df_penjelasan[['no_pasal', 'penjelasan_pasal', 'no_ayat', 'penjelasan']]

In [23]:
df_penjelasan

,no_pasal,penjelasan_pasal,no_ayat,penjelasan
0,1,Cukup jelas.,,Cukup jelas.
0,2,Cukup jelas.,,Cukup jelas.
0,3,Cukup jelas.,,Cukup jelas.
0,4,Cukup jelas.,,Cukup jelas.
0,5,Huruf a\nCukup jelas.\nHuruf b\nCukup jelas.\n...,,Huruf a\nCukup jelas.\nHuruf b\nCukup jelas.\n...
...,...,...,...,...
0,83,Cukup jelas.,,Cukup jelas.
0,84,Cukup jelas.,,Cukup jelas.
0,85,Cukup jelas.,,Cukup jelas.
0,86,Cukup jelas.,,Cukup jelas.


In [24]:
df_badan[df_badan['no_pasal'] == '10']

,no_bab,nama_bab,no_bagian,nama_bagian,no_paragraf,nama_paragraf,no_pasal,no_ayat,isi_ayat
0,IV,PENDIRIAN,,,,,10,1,Ormas sebagaimana dimaksud dalam Pasal 9 dapa...
1,IV,PENDIRIAN,,,,,10,2,Ormas sebagaimana dimaksud pada ayat (1) dapa...


In [25]:
penjelasan

'TAMBAHAN\nLEMBARAN NEGARA RI\nNo. 5430 HAK ASASI MANUSIA. Organisasi\nKemasyarakatan. Pendirian-Pengawasan.\nPencabutan. (Penjelasan Atas Lembaran Negara\nRepublik Indonesia Tahun 2013 Nomor 116)\nPENJELASAN\nATAS\nUNDANG-UNDANG REPUBLIK INDONESIA\nNOMOR 17 TAHUN 2013\nTENTANG\nORGANISASI KEMASYARAKATAN\nI. UMUM\nUndang-Undang Dasar Negara Republik Indonesia Tahun 1945\nmenjamin kemerdekaan berserikat, berkumpul, dan mengeluarkan\npendapat serta memajukan dirinya dalam memperjuangkan haknya\nsecara individu ataupun kolektif untuk membangun masyarakat,\nbangsa, dan Negara Kesatuan Republik Indonesia sebagai perwujudan\nhak asasi manusia. Pasal 28J ayat (2) Undang-Undang Dasar Negara\nRepublik Indonesia Tahun 1945 menyebutkan bahwa dalam\nmenjalankan hak asasi dan kebebasannya secara individu maupun\nkolektif, setiap orang wajib menghormati hak asasi manusia lainnya\ndan wajib tunduk kepada pembatasan yang ditetapkan undang-\nundang dengan maksud semata-mata untuk menjamin pengakuan\nse

In [26]:
df_penjelasan['pasal_ayat'] = df_penjelasan['no_pasal'] + "-" + df_penjelasan['no_ayat']
df_badan['pasal_ayat'] = df_badan['no_pasal'] + "-" + df_badan['no_ayat']

In [27]:
df_left = df_badan.merge(df_penjelasan, how="left", on="pasal_ayat", suffixes=('', '_y'))
df_left = df_left.drop(['pasal_ayat'], axis=1)

In [28]:
df_left

,no_bab,nama_bab,no_bagian,nama_bagian,no_paragraf,nama_paragraf,no_pasal,no_ayat,isi_ayat,no_pasal_y,penjelasan_pasal,no_ayat_y,penjelasan
0,I,KETENTUAN UMUM,,,,,1,,Dalam Undang-Undang ini yang dimaksud dengan:\...,1,Cukup jelas.,,Cukup jelas.
1,II,"ASAS, CIRI, DAN SIFAT",,,,,2,,Asas Ormas tidak bertentangan dengan Pancasila...,2,Cukup jelas.,,Cukup jelas.
2,II,"ASAS, CIRI, DAN SIFAT",,,,,3,,Ormas dapat mencantumkan ciri tertentu yang me...,3,Cukup jelas.,,Cukup jelas.
3,II,"ASAS, CIRI, DAN SIFAT",,,,,4,,"Ormas bersifat sukarela, sosial, mandiri, nirl...",4,Cukup jelas.,,Cukup jelas.
4,III,"TUJUAN, FUNGSI, DAN RUANG LINGKUP",,,,,5,,Ormas bertujuan untuk:\na. meningkatkan partis...,5,Huruf a\nCukup jelas.\nHuruf b\nCukup jelas.\n...,,Huruf a\nCukup jelas.\nHuruf b\nCukup jelas.\n...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,XVIII,KETENTUAN PERALIHAN,,,,,83,,Pada saat Undang Undang ini mulai berlaku:\na....,83,Cukup jelas.,,Cukup jelas.
183,XIX,KETENTUAN PENUTUP,,,,,84,,"Pada saat Undang-Undang ini mulai berlaku, sem...",84,Cukup jelas.,,Cukup jelas.
184,XIX,KETENTUAN PENUTUP,,,,,85,,"Pada saat Undang-Undang ini mulai berlaku, Und...",85,Cukup jelas.,,Cukup jelas.
185,XIX,KETENTUAN PENUTUP,,,,,86,,Peraturan pelaksanaan dari Undang-Undang ini h...,86,Cukup jelas.,,Cukup jelas.


In [29]:
df_left[pd.isna(df_left['penjelasan'])]

,no_bab,nama_bab,no_bagian,nama_bagian,no_paragraf,nama_paragraf,no_pasal,no_ayat,isi_ayat,no_pasal_y,penjelasan_pasal,no_ayat_y,penjelasan
6,III,"TUJUAN, FUNGSI, DAN RUANG LINGKUP",,,,,7,1,Ormas memiliki bidang kegiatan sesuai dengan ...,NaN,NaN,NaN,NaN
7,III,"TUJUAN, FUNGSI, DAN RUANG LINGKUP",,,,,7,2,Bidang kegiatan sebagaimana dimaksud pada aya...,NaN,NaN,NaN,NaN
10,IV,PENDIRIAN,,,,,10,1,Ormas sebagaimana dimaksud dalam Pasal 9 dapa...,NaN,NaN,NaN,NaN
11,IV,PENDIRIAN,,,,,10,2,Ormas sebagaimana dimaksud pada ayat (1) dapa...,NaN,NaN,NaN,NaN
12,IV,PENDIRIAN,,,,,11,1,Ormas berbadan hukum sebagaimana dimaksud dal...,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
174,XVII,SANKSI,,,,,77,2,Permohonan kasasi sebagaimana dimaksud dalam ...,NaN,NaN,NaN,NaN
175,XVII,SANKSI,,,,,78,1,Panitera Mahkamah Agung wajib menyampaikan sa...,NaN,NaN,NaN,NaN
176,XVII,SANKSI,,,,,78,2,Pengadilan negeri wajib menyampaikan salinan ...,NaN,NaN,NaN,NaN
179,XVII,SANKSI,,,,,81,1,Setiap orang yang merupakan anggota atau peng...,NaN,NaN,NaN,NaN


In [30]:
df_right = df_badan.merge(df_penjelasan, how="right", on="pasal_ayat", suffixes=('', '_y'))
df_right = df_right.drop(['pasal_ayat'], axis=1)

In [31]:
df_right

,no_bab,nama_bab,no_bagian,nama_bagian,no_paragraf,nama_paragraf,no_pasal,no_ayat,isi_ayat,no_pasal_y,penjelasan_pasal,no_ayat_y,penjelasan
0,I,KETENTUAN UMUM,,,,,1,,Dalam Undang-Undang ini yang dimaksud dengan:\...,1,Cukup jelas.,,Cukup jelas.
1,II,"ASAS, CIRI, DAN SIFAT",,,,,2,,Asas Ormas tidak bertentangan dengan Pancasila...,2,Cukup jelas.,,Cukup jelas.
2,II,"ASAS, CIRI, DAN SIFAT",,,,,3,,Ormas dapat mencantumkan ciri tertentu yang me...,3,Cukup jelas.,,Cukup jelas.
3,II,"ASAS, CIRI, DAN SIFAT",,,,,4,,"Ormas bersifat sukarela, sosial, mandiri, nirl...",4,Cukup jelas.,,Cukup jelas.
4,III,"TUJUAN, FUNGSI, DAN RUANG LINGKUP",,,,,5,,Ormas bertujuan untuk:\na. meningkatkan partis...,5,Huruf a\nCukup jelas.\nHuruf b\nCukup jelas.\n...,,Huruf a\nCukup jelas.\nHuruf b\nCukup jelas.\n...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,XVIII,KETENTUAN PERALIHAN,,,,,83,,Pada saat Undang Undang ini mulai berlaku:\na....,83,Cukup jelas.,,Cukup jelas.
92,XIX,KETENTUAN PENUTUP,,,,,84,,"Pada saat Undang-Undang ini mulai berlaku, sem...",84,Cukup jelas.,,Cukup jelas.
93,XIX,KETENTUAN PENUTUP,,,,,85,,"Pada saat Undang-Undang ini mulai berlaku, Und...",85,Cukup jelas.,,Cukup jelas.
94,XIX,KETENTUAN PENUTUP,,,,,86,,Peraturan pelaksanaan dari Undang-Undang ini h...,86,Cukup jelas.,,Cukup jelas.


In [32]:
df_right[pd.isna(df_right['isi_ayat'])]

,no_bab,nama_bab,no_bagian,nama_bagian,no_paragraf,nama_paragraf,no_pasal,no_ayat,isi_ayat,no_pasal_y,penjelasan_pasal,no_ayat_y,penjelasan
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,Cukup jelas.,,Cukup jelas.
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,Cukup jelas.,,Cukup jelas.
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11,Cukup jelas.,,Cukup jelas.
11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12,Cukup jelas.,,Cukup jelas.
13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14,Cukup jelas.,,Cukup jelas.
14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15,Cukup jelas.,,Cukup jelas.
15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16,Cukup jelas.,,Cukup jelas.
16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17,Cukup jelas.,,Cukup jelas.
17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18,Cukup jelas.,,Cukup jelas.
28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29,Cukup jelas.,,Cukup jelas.


In [33]:
df = df_badan.merge(df_penjelasan[['no_pasal', 'penjelasan_pasal']], how="inner", on="no_pasal", suffixes=('', '_y')).drop_duplicates()
df = df.merge(df_penjelasan[['pasal_ayat', 'no_ayat', 'penjelasan']], how="left", on="pasal_ayat", suffixes=('', '_y'))
# df = df.drop(['pasal_ayat', 'no_pasal_y', 'no_ayat_y', 'penjelasan_pasal'], axis=1)

In [34]:
df

,no_bab,nama_bab,no_bagian,nama_bagian,no_paragraf,nama_paragraf,no_pasal,no_ayat,isi_ayat,pasal_ayat,penjelasan_pasal,no_ayat_y,penjelasan
0,I,KETENTUAN UMUM,,,,,1,,Dalam Undang-Undang ini yang dimaksud dengan:\...,1-,Cukup jelas.,,Cukup jelas.
1,II,"ASAS, CIRI, DAN SIFAT",,,,,2,,Asas Ormas tidak bertentangan dengan Pancasila...,2-,Cukup jelas.,,Cukup jelas.
2,II,"ASAS, CIRI, DAN SIFAT",,,,,3,,Ormas dapat mencantumkan ciri tertentu yang me...,3-,Cukup jelas.,,Cukup jelas.
3,II,"ASAS, CIRI, DAN SIFAT",,,,,4,,"Ormas bersifat sukarela, sosial, mandiri, nirl...",4-,Cukup jelas.,,Cukup jelas.
4,III,"TUJUAN, FUNGSI, DAN RUANG LINGKUP",,,,,5,,Ormas bertujuan untuk:\na. meningkatkan partis...,5-,Huruf a\nCukup jelas.\nHuruf b\nCukup jelas.\n...,,Huruf a\nCukup jelas.\nHuruf b\nCukup jelas.\n...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,XVIII,KETENTUAN PERALIHAN,,,,,83,,Pada saat Undang Undang ini mulai berlaku:\na....,83-,Cukup jelas.,,Cukup jelas.
183,XIX,KETENTUAN PENUTUP,,,,,84,,"Pada saat Undang-Undang ini mulai berlaku, sem...",84-,Cukup jelas.,,Cukup jelas.
184,XIX,KETENTUAN PENUTUP,,,,,85,,"Pada saat Undang-Undang ini mulai berlaku, Und...",85-,Cukup jelas.,,Cukup jelas.
185,XIX,KETENTUAN PENUTUP,,,,,86,,Peraturan pelaksanaan dari Undang-Undang ini h...,86-,Cukup jelas.,,Cukup jelas.


In [35]:
df_left.shape

(187, 13)

In [36]:
df.shape

(187, 13)

In [37]:
def ambil_penjelasan_pasal_2(text, no_pasal):
    pattern = f"(?<=\\n|^|\s)(Pasal\s*{no_pasal}\s*\\n.*?)(?=\\nPasal\s*{no_pasal}\s*\\n|\sPasal\s*{no_pasal}\s*\\n|$)"
    # pattern = "(?<=\\n)(Pasal\s*[\p{N}IVXLCDM]+\s*\\n.*?)(?=\\nPasal\s*\p{N}+\s*\\n|$)"
    pasal = re.findall(pattern, text, re.DOTALL)
    if pasal:
        pattern_no = "(?<=Pasal)(?:\s*)([\p{N}IVXLCDM]+)"
        pattern_isi = "(?<=Pasal)(?:\s*[\p{N}IVXLCDM]+\s*\\n)(.*)"
        no = [re.search(pattern_no, i, re.DOTALL).group(1) if re.search(pattern_no, i, re.DOTALL) else "ERROR" for i in pasal][0]
        isi = [re.search(pattern_isi, i, re.DOTALL).group(1) if re.search(pattern_isi, i, re.DOTALL) else "ERROR" for i in pasal][0]
        return no, isi
    else:
        no = [""]
        isi = [text]
        return no, isi

In [38]:
def ambil_penjelasan_ayat_2(text, no_ayat):
    pattern = f"(?<=\\n|^|\s)(Ayat\s*\({no_ayat}\).*?)(?=\\nAyat\s*\({no_ayat}\)|\sAyat\s*\({no_ayat}\)|$)"
    # pattern = "(?<=\\n|^)(\s*Ayat\s*\(\p{N}+\).*?)(?=\\nAyat\s*\(\p{N}+\)|$)"
    ayat = re.findall(pattern, text, re.DOTALL)
    if ayat:
        pattern_no = "(?<=Ayat)(?:\s*\()(\p{N}+)"
        pattern_isi = "(?<=Ayat)(?:\s*\(\p{N}+\)\s*\\n)(.*)"
        no = [re.search(pattern_no, i, re.DOTALL).group(1) if re.search(pattern_no, i, re.DOTALL) else "ERROR" for i in ayat]
        isi = [re.search(pattern_isi, i, re.DOTALL).group(1) if re.search(pattern_isi, i, re.DOTALL) else "ERROR" for i in ayat]
        return no, isi
    else:
        no = [""][0]
        isi = [text][0]
        return no, isi

In [39]:
# missing_penjelasan_pasal = list(df_left[pd.isna(df_left['penjelasan_pasal'])]['no_pasal'])
# missing_penjelasan_ayat = list(df_left[pd.isna(df_left['penjelasan'])]['no_ayat'])

In [40]:
# for pasal in missing_penjelasan_pasal:
#     no_pasal, penjelasan_pasal = ambil_penjelasan_pasal_2(df_left[df_left['no_pasal'] == str(int(pasal) - 1)]['penjelasan_pasal'].drop_duplicates().values[0], pasal)
#     df_left.loc[df_left['no_pasal'] == pasal, 'penjelasan_pasal'] = penjelasan_pasal
#     df_left.loc[df_left['no_pasal'] == pasal, 'no_pasal_y'] = no_pasal

# for pasal, ayat in zip(missing_penjelasan_pasal, missing_penjelasan_ayat):
#     no_ayat, penjelasan_ayat = ambil_penjelasan_ayat_2(df_left[(df_left['no_pasal'] == pasal) & (df_left['no_ayat'] == ayat)]['penjelasan_pasal'].drop_duplicates().values[0], ayat)
#     df_left.loc[(df_left['no_pasal'] == pasal) & (df_left['no_ayat'] == ayat), 'penjelasan'] = penjelasan_ayat
#     df_left.loc[(df_left['no_pasal'] == pasal) & (df_left['no_ayat'] == ayat), 'no_ayat_y'] = no_ayat

In [41]:
# df_left[df_left['no_pasal'] == '5']

## UU YANG SUDAH BISA
UU1-1999<br>
UU6-1999 (Pasalnya pake romawi) FIXED<br>
UU7-1999 (Gaada penjelasan) FIXED<br>
UU8-1999<br>
UU9-1999<br>
UU10-1999<br>
UU11-1999<br>
UU13-1999<br>
UU19-1999<br>
UU21-1999<br>

## UU YANG BELOM BISA
UU12-1999 (Penjelasan gaada ayat)<br>
UU15-1999 (Nomor ayat salah, penjelasan untuk 2 ayat kurung 2 kali)<br>
UU16-1999 (Badan gaada ayat)<br>
UU20-1999 (Penjelasan pasal ada titiknya)<br>
UU22-1999 (Penjelasan gaada ayat)<br>